## 0. Описание лабораторной работы
**Задание:**

Разработать и оценить классификатор, прогнозирующий вероятность прекращения сотрудничества клиента с банком. Основное требование — добиться F1-меры не ниже 0.59 на тестовом наборе. Для дополнительного анализа качества использовать метрику ROC AUC.

В распоряжении имеются данные с информацией о клиентах финансовой компании, включающие:
- **Number** — индекс строки в данных
- **CustomerId** — уникальный идентификатор клиента
- **Surname** — фамилия
- **CreditScore** — кредитный рейтинг
- **Geography** — страна проживания
- **Gender** — пол
- **Age** — возраст
- **Tenure** — сколько лет человек является клиентом банка
- **Balance** — баланс на счёте
- **NumOfProducts** — количество продуктов банка, используемых клиентом
- **HasCrCard** — наличие кредитной карты
- **IsActiveMember** — активность клиента
- **EstimatedSalary** — предполагаемая зарплата
- **Exited** — факт ухода клиента


**Этапы выполнения:**

1. Работа с данными
    * Загрузить набор, изучить формат и виды переменных.
    * Провести очистку — убрать или скорректировать дубликаты, аномалии и пропуски.
    * Проанализировать распределение целевой переменной для выявления дисбаланса.
2. Обучение эталонных моделей
    * Выполнить разбиение исходных данных на обучающий и контрольный наборы.
    * Обучить базовую модель без учета дисбаланса, например, логистическую регрессию либо дерево решений.
    * Рассчитать базовые метрики качества: F1-меру и ROC AUC, проанализировать результаты.
3. Оптимизация с учетом дисбаланса
    * Применить техники балансировки классов, такие как взвешивание, изменения в обучающем наборе (oversampling/undersampling).
    * Обучить несколько алгоритмов (2 и более).
    * Найти и зафиксировать модель с максимальным значением F1-меры.
4. Итоговое тестирование и сравнение
    * Оцените финальную модель на независимом тестовом наборе.
    * Сравните F1-меру с ROC AUC, обсудите взаимосвязь и практическую значимость.
    * Сделайте рекомендации по использованию модели в бизнес-процессах.



In [ ]:
import warnings
warnings.filterwarnings('ignore')
import plotly.io as pio
pio.templates.default = 'plotly_white'

import polars as pl
import numpy as np
import random
import sklearn
import os

seed = 42
random.seed(seed)
np.random.seed(seed)
sklearn.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

## 1. Работа с данными


Продолжу <s>мучать себя</s> пользоваться решением `polars`.

Для совместимости с `sklearn` придется иногда конвертировать датафрейм в `pd.DataFrame`, но это проблема будущего меня.

*Я в настоящем выбрал быть счастливым.*

In [ ]:
data_url = 'https://github.com/Lopa10ko/itmo-ml-2025/raw/main/lab-2/Bank_data.csv'
df_pl = pl.read_csv(data_url)

print(f'Размер датасета: {df_pl.shape}\n')

Размер датасета: (10000, 14)



Сначала всегда полезно посмотреть на грязные данные.

In [ ]:
display(df_pl.head())

print(f'\n\nОписательная статистика:')
display(df_pl.describe())


RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
i64,i64,str,i64,str,str,i64,f64,f64,i64,i64,i64,f64,i64
1,15634602,"""Hargrave""",619,"""France""","""Female""",42,2.0,0.0,1,1,1,101348.88,1
2,15647311,"""Hill""",608,"""Spain""","""Female""",41,1.0,83807.86,1,0,1,112542.58,0
3,15619304,"""Onio""",502,"""France""","""Female""",42,8.0,159660.8,3,1,0,113931.57,1
4,15701354,"""Boni""",699,"""France""","""Female""",39,1.0,0.0,2,0,0,93826.63,0
5,15737888,"""Mitchell""",850,"""Spain""","""Female""",43,2.0,125510.82,1,1,1,79084.1,0




Описательная статистика:


statistic,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
str,f64,f64,str,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",10000.0,10000.0,"""10000""",10000.0,"""10000""","""10000""",10000.0,9091.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
"""null_count""",0.0,0.0,"""0""",0.0,"""0""","""0""",0.0,909.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",5000.5,1.5691e7,null,650.5288,null,null,38.9218,4.99769,76485.889288,1.5302,0.7055,0.5151,100090.239881,0.2037
"""std""",2886.89568,71936.186123,null,96.653299,null,null,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
"""min""",1.0,1.5565701e7,"""Abazu""",350.0,"""France""","""Female""",18.0,0.0,0.0,1.0,0.0,0.0,11.58,0.0
"""25%""",2501.0,1.562853e7,null,584.0,null,null,32.0,2.0,0.0,1.0,0.0,0.0,51011.29,0.0
"""50%""",5001.0,1.5690743e7,null,652.0,null,null,37.0,5.0,97208.46,1.0,1.0,1.0,100200.4,0.0
"""75%""",7500.0,1.5753229e7,null,718.0,null,null,44.0,7.0,127642.44,2.0,1.0,1.0,149384.43,0.0
"""max""",10000.0,1.581569e7,"""Zuyeva""",850.0,"""Spain""","""Male""",92.0,10.0,250898.09,4.0,1.0,1.0,199992.48,1.0


Пока понятно, что данные в этой лабораторной работе имеют не так уж и много пропусков, что не может не радовать.

1. Индекс `RowNumber` нам ни о чем полезном не говорит и сказать не может, ровно как и `CustomerId`. Эти два столбца стоит лишь проверить на наличие дубликатов по этим индексам. Хотя судя по статистикам `RowNumber` - это просто действительно "номер кортежа" (ничего себе).

2. Фамилия (`Surname`) с первого взгляда тоже не кажется чем-то интересным, но нужно помнить о том, что бывают однофамильцы (одинаковые фамилии с разными айдишниками), а бывают семьи (тоже самое), а могут быть и клиенты, которые успели стать клиентами банка несколько раз за временной период (интересно, что в некоторых задачах это становится важно).

3. Кредитный рейтинг (`CreditScore`) пока остается для меня загадкой, так как я, не будучи экспертом, не знаю, что он из себя представляет.

4. Интересно посмотреть на признак несуществующих стран в (`Geography`) и на бинарность/небинарность признака гендера (шучу!).

5. Единственная фича с наличием пропусков в данных "преданность банку" `Tenure` (сколько лет он является клиентом банка).

6. Категориальная фича `NumOfProducts` (сколькими продуктами банка пользуется клиент) кажется сильно несбалансированной, но это мы еще посмотрим.

7. В бинарных фичах наличия кредитной карты `HasCrCard` и "активен ли клиент" `IsActiveMember` нужно тоже посмотреть баланс классов.

8. Ну и непосредственно бинарный таргет "ушел ли клиент" `Exited` нужно тоже проанализировать на дисбаланс.

Еще раз, пожалуй, выведем информацию о типах данных и количестве пропусков:

In [ ]:
for i, col in enumerate(df_pl.columns):
    dtype = df_pl[col].dtype
    null_count = df_pl[col].null_count()
    print(f'{i + 1}) {col} ({dtype}), missing: {null_count}')

1) RowNumber (Int64), missing: 0
2) CustomerId (Int64), missing: 0
3) Surname (String), missing: 0
4) CreditScore (Int64), missing: 0
5) Geography (String), missing: 0
6) Gender (String), missing: 0
7) Age (Int64), missing: 0
8) Tenure (Float64), missing: 909
9) Balance (Float64), missing: 0
10) NumOfProducts (Int64), missing: 0
11) HasCrCard (Int64), missing: 0
12) IsActiveMember (Int64), missing: 0
13) EstimatedSalary (Float64), missing: 0
14) Exited (Int64), missing: 0


## 2. Предобработка данных


### 2.1 Общий анализ данных

Показываем уникальные значения для (потенциально) категориальных признаков.

*Эвристика: если уникальных значений признака меньше 40, то стоит задуматься о том, что он категориальный.*

In [ ]:
from prettytable import PrettyTable

uniques_tb = PrettyTable()
uniques_tb.field_names = ['feature', 'n_unique', 'Values (if quantity less than 40)']
category_like_columns = []

for col in df_pl.columns:
    unique_count = df_pl[col].n_unique()
    if unique_count < 40:
        category_like_columns.append(col)
        unique_values = df_pl[col].drop_nulls().unique().sort().to_list()
        values_str = ', '.join(map(str, unique_values))
    else:
        values_str = '-'

    uniques_tb.add_row([col, unique_count, values_str])

print(uniques_tb)

+-----------------+----------+--------------------------------------------------------+
|     feature     | n_unique |           Values (if quantity less than 40)            |
+-----------------+----------+--------------------------------------------------------+
|    RowNumber    |  10000   |                           -                            |
|    CustomerId   |  10000   |                           -                            |
|     Surname     |   2932   |                           -                            |
|   CreditScore   |   460    |                           -                            |
|    Geography    |    3     |                 France, Germany, Spain                 |
|      Gender     |    2     |                      Female, Male                      |
|       Age       |    70    |                           -                            |
|      Tenure     |    12    | 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0 |
|     Balance     |   6382   |  

Вот и думайте! Количество уникальных значений в индексах совпадает с размерностью датасета, поэтому нужно будет избавиться от этих недо-признаков в будущем, а вот фамилии у многих посовпадали. Видимо, банк-то семейный!

Кажется, что возраст странно было бы посчитать категориальным, так как в целом <s>время не щадит никого</s> и значение при желании можно посчитать "на сегодняшний день", зная сегодняшнюю дату и дату рождения клиента.
Такое оправдание я придумал, чтобы прикрыть факт того, что не хочу работать с 70 классами одного признака.

Все, что в итоге вывелось в таблицу, запомним и аккуратно положим в список названий категориальных фичей.

Теперь выведем таблицы для каждого (потенциально) категориального признака и посмотрим распределение сэмплов по каждому из значений. Дальше при обработке я буду руководствоваться похожими агрегатами, чтобы удалять из сэмплов нерепрезентативные классы.

In [ ]:
from prettytable import PrettyTable

def print_unique_values_table(df, category):
    category_counts = df.group_by(category).agg(pl.count().alias('count')).sort('count', descending=True)
    table = PrettyTable()
    table.field_names = [category, 'count', 'percent (%)']

    total_count = category_counts['count'].sum()
    for row in category_counts.iter_rows(named=True):
        value = row[category]
        count = row['count']
        percentage = (count / total_count) * 100
        table.add_row([value, count, f'{percentage:.1f}%'])

    print(table)

for category in category_like_columns:
    print_unique_values_table(df_pl, category)

+-----------+-------+-------------+
| Geography | count | percent (%) |
+-----------+-------+-------------+
|   France  |  5014 |    50.1%    |
|  Germany  |  2509 |    25.1%    |
|   Spain   |  2477 |    24.8%    |
+-----------+-------+-------------+
+--------+-------+-------------+
| Gender | count | percent (%) |
+--------+-------+-------------+
|  Male  |  5457 |    54.6%    |
| Female |  4543 |    45.4%    |
+--------+-------+-------------+
+--------+-------+-------------+
| Tenure | count | percent (%) |
+--------+-------+-------------+
|  1.0   |  952  |     9.5%    |
|  2.0   |  950  |     9.5%    |
|  8.0   |  933  |     9.3%    |
|  3.0   |  928  |     9.3%    |
|  5.0   |  927  |     9.3%    |
|  7.0   |  925  |     9.2%    |
|  None  |  909  |     9.1%    |
|  4.0   |  885  |     8.8%    |
|  9.0   |  882  |     8.8%    |
|  6.0   |  881  |     8.8%    |
|  10.0  |  446  |     4.5%    |
|  0.0   |  382  |     3.8%    |
+--------+-------+-------------+
+---------------+-----

Из этих таблиц видно, что:

1. Место проживания `Geography` - несбалансированный признак, но хорошо в целом определяет кластеры кортежей (ну тут все даже звучит логично, так как в разных странах экономическая обстановка должна быть специфической). Каждый второй клиент любит есть лягушек, ну что же, не осуждаем.
2. Гендерный признак `Gender` и `IsActiveMember` хорошо сбалансированы, множество явно разделимо по каждому из этих признаков.
3. Преданность клиента банку `Tenure` неплохо сбалансирована, поэтому здесь предлагается пропуски просто заполнить случайными известными значениями, чтобы не менять характер распределения и просто немного "зашумить" данные.
4. В `NumOfProducts`, кажется, что можно просто удалить сэмплы, когда клиент пользуется 3 и 4 продуктами банка, так как сами эти классы недостаточно мощны, чтобы быть репрезентативными.
5. Признак `HasCrCard` несбалансирован, ровно как и таргетный признак `Exited`.

### 2.2 Обработка пропущенных значений


Вновь посмотрим, сколько пропущенных значений в каждом столбце:

In [ ]:
import plotly.express as px


def analyze_missing_values_by_column(df):
    total_rows = df.height

    missing_analysis = df.select([
        pl.col(col).null_count().alias(f'{col}_missing') for col in df.columns
    ]).transpose(include_header=True, header_name='column', column_names=['missing_count'])

    missing_df = missing_analysis.with_columns([
        (pl.col('missing_count') / total_rows * 100).alias('missing_percent')
    ]).sort('missing_percent', descending=True)

    missing_with_nulls = missing_df.filter(pl.col('missing_count') > 0)
    if len(missing_with_nulls) == 0:
        return "No more missing values!"

    print(missing_with_nulls)

    missing_data = {
        'column': missing_with_nulls['column'].to_list(),
        'missing_percent': missing_with_nulls['missing_percent'].to_list()
    }
    fig = px.bar(
        missing_data,
        x='column',
        y='missing_percent',
        title='Missing values percent by column',
        labels={'missing_percent': 'Missing values percent', 'column': 'columns'},
        color='missing_percent',
        color_continuous_scale='Reds'
    )
    fig.update_layout(
        xaxis_tickangle=-45,
        height=500,
        showlegend=False
    )
    fig.show()

analyze_missing_values_by_column(df_pl)

shape: (1, 3)
┌────────────────┬───────────────┬─────────────────┐
│ column         ┆ missing_count ┆ missing_percent │
│ ---            ┆ ---           ┆ ---             │
│ str            ┆ u32           ┆ f64             │
╞════════════════╪═══════════════╪═════════════════╡
│ Tenure_missing ┆ 909           ┆ 9.09            │
└────────────────┴───────────────┴─────────────────┘


Стратегия заполнения пропущенных значений:

In [ ]:
df_processed = df_pl.clone()

Категориальная фича `Tenure` является признаком с относительно низким значением процента пропусков (~10%), поэтому можно было бы заполнить просто модой, но такой подход создаст пики распределения, поэтому здесь ничего лучше не придумал, как заполнять пропуски случайными известными значениями этого атрибута.
Да, это зашумит распределение, но все равно, кажется, звучит логично.

In [ ]:
column = 'Tenure'
existing_values = df_processed[column].drop_nulls().to_list()
missing_count = df_processed[column].null_count()

if missing_count > 0:
    rng = np.random.default_rng(seed=seed)
    random_values = rng.choice(existing_values, size=missing_count, replace=True)
    null_indices = df_processed.select(
        pl.int_range(0, pl.len()).filter(pl.col(column).is_null())
    ).to_series().to_list()

    new_column = df_processed[column].to_list()
    for idx, replacement in zip(null_indices, random_values):
        new_column[idx] = replacement

    df_processed = df_processed.with_columns(pl.Series(column, new_column))

Сравним распределения `Tenure` до и после заполнения пропусков случайными известными значениями.

In [ ]:
from scipy.stats import gaussian_kde
import numpy as np
import plotly.graph_objects as go

def plot_distribution_comparison(df_new, df_old, column):
    data1 = df_new.filter(pl.col(column).is_not_null())[column]
    kde1 = gaussian_kde(data1)
    x_range1 = np.linspace(data1.min(), data1.max(), 100)
    y_range1 = kde1(x_range1)

    data2 = df_old.filter(pl.col(column).is_not_null())[column]
    kde2 = gaussian_kde(data2)
    x_range2 = np.linspace(data2.min(), data2.max(), 100)
    y_range2 = kde2(x_range2)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_range1, y=y_range1, mode='lines',
                            name='KDE (new)', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=x_range2, y=y_range2, mode='lines',
                            name='KDE (old)', line=dict(color='red')))

    fig.add_trace(go.Histogram(
        x=data1,
        histnorm='probability density',
        opacity=0.3,
        name='histogram (new)',
        marker_color='blue'
    ))
    fig.add_trace(go.Histogram(
        x=data2,
        histnorm='probability density',
        opacity=0.3,
        name='histogram(old)',
        marker_color='red'
    ))

    fig.update_layout(
        xaxis_title=f'{column} values',
        yaxis_title='probability density',
        barmode='overlay'
    )
    fig.update_traces(opacity=0.6)
    fig.show()

plot_distribution_comparison(df_processed, df_pl, 'Tenure')

А вот, что получилось бы, если бы мы заменили пропуски в `Tenure` модой:

In [ ]:
tenure_median = df_pl['Tenure'].mode()
df_badly_processed = df_pl.with_columns(
    pl.col('Tenure').fill_null(tenure_median)
)
plot_distribution_comparison(df_badly_processed, df_pl, 'Tenure')

Ну вот и все, кажется, от пропусков избавились.

In [ ]:
analyze_missing_values_by_column(df_processed)

'No more missing values!'

In [ ]:
df_processed.describe()

statistic,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
str,f64,f64,str,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",10000.0,10000.0,"""10000""",10000.0,"""10000""","""10000""",10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0
"""null_count""",0.0,0.0,"""0""",0.0,"""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",5000.5,1.5691e7,null,650.5288,null,null,38.9218,5.0106,76485.889288,1.5302,0.7055,0.5151,100090.239881,0.2037
"""std""",2886.89568,71936.186123,null,96.653299,null,null,10.487806,2.898711,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
"""min""",1.0,1.5565701e7,"""Abazu""",350.0,"""France""","""Female""",18.0,0.0,0.0,1.0,0.0,0.0,11.58,0.0
"""25%""",2501.0,1.562853e7,null,584.0,null,null,32.0,2.0,0.0,1.0,0.0,0.0,51011.29,0.0
"""50%""",5001.0,1.5690743e7,null,652.0,null,null,37.0,5.0,97208.46,1.0,1.0,1.0,100200.4,0.0
"""75%""",7500.0,1.5753229e7,null,718.0,null,null,44.0,8.0,127642.44,2.0,1.0,1.0,149384.43,0.0
"""max""",10000.0,1.581569e7,"""Zuyeva""",850.0,"""Spain""","""Male""",92.0,10.0,250898.09,4.0,1.0,1.0,199992.48,1.0


### 2.3 Обработка дубликатов


In [ ]:
df_deduplicated = df_processed.clone()

Очевидно, что полных дубликатов в семплах уже не будет, так как после удаления или заполнения пропусков в каких-то фичах я таким образом мог сам себя обмануть и добавить зашумленность в данные (например, в стратегии заполнения случайными известными значениям).

In [ ]:
initial_count = df_deduplicated.height
df_unique = df_deduplicated.unique()
duplicates = initial_count - df_unique.height
print(f'Full duplicates: {duplicates}')

Full duplicates: 0


Проверим дубликаты по всем характеристикам, в которые я не добавлял случайные значения, так как я думаю, что могут сущестовать клиенты, которые в данных представлены несколько раз.

In [ ]:
print(f'Shape before: {df_deduplicated.shape}')
key_features = ['Geography', 'Gender', 'Surname', 'Age', 'Exited']
df_unique_key = df_deduplicated.unique(subset=key_features)
duplicates_key = initial_count - df_unique_key.height
print(f'Key-feature duplicates: {duplicates_key}')
df_deduplicated = df_unique_key
print(f'Shape after: {df_deduplicated.shape}')

Shape before: (10000, 14)
Key-feature duplicates: 190
Shape after: (9810, 14)


Попались!

### 2.4 Обработка аномальных значений и выбросов


In [ ]:
df_cleared = df_deduplicated.clone()

Для начала нужно в целом посмотреть на распределения и выбросы в ключевых вещественных признаках:
- `CreditScore`
- `Age`
- `Balance`
- `EstimatedSalary`

In [ ]:
from plotly.subplots import make_subplots

key_numeric_features = ['CreditScore',
                        'Age',
                        'Balance',
                        'EstimatedSalary']

fig = make_subplots(rows=1, cols=4, subplot_titles=key_numeric_features, vertical_spacing=0.2)

for i, col in enumerate(key_numeric_features):
    fig.add_trace(
        go.Box(
            y=df_cleared[col],
            name=col,
            showlegend=False,
            boxpoints='outliers'
        ),
        row=1, col=i+1
    )
fig.update_layout(height=1000, width=1500, showlegend=False)
fig.show()

Взглянем также на распределение каждого признака и соответсвующее ему нормальное. То есть при логнормальном распределении признака достаточно будет логарифмировать.

На самом деле это очень важный этап, так как при удалении выбросов методом IQR или Z-методом в логнормальном распределении нужно учитывать асимметрию.
А в мультимодальном такие методы просто "размажут" моды.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def diagnostic_plots(df, columns):
    subplot_titles = []
    subplot_titles
    fig = make_subplots(
        rows=len(columns), cols=2,
        subplot_titles = [f'{col} ({suffix})' for col in columns for suffix in ['original', 'log']]
    )

    for i, col in enumerate(columns):
        fig.add_trace(
            go.Histogram(x=df[col], name=col, showlegend=False),
            row=i+1, col=1
        )
        if df[col].min() > 0:
            log_data = np.log1p(df[col])
            fig.add_trace(
                go.Histogram(x=log_data, name=f'log({col})', showlegend=False),
                row=i+1, col=2
            )

    fig.update_layout(height=300*len(columns))
    fig.show()

diagnostic_plots(df_cleared, key_numeric_features)

Ага, то есть видны неожиданные (или ожидаемые) пики оригинальных распределений
1. `Balance` около трети всех клиентов не больше 2500 единиц
2. Матожидание `Age` смещено.
3. По признаку `CreditScore` есть подмножество клиентов с достаточно высоким значением.
4. Признак `EstimatedSalary`, кажется, распределен равномерно. И при его логарифмировании получаем характерное "показательное" очертание.



Воспользуюсь детектором распределений и уберу хвосты автоматически.

In [ ]:
!pip install pyod --quiet

In [ ]:
from pyod.models.ecod import ECOD
import polars as pl
import numpy as np

def ecod_clean_dataset(df, contamination=0.2):
    original_indices = np.arange(len(df))
    numeric_cols = df.select(pl.col(pl.NUMERIC_DTYPES)).columns
    X = df[numeric_cols].to_numpy()

    detector = ECOD(contamination=contamination)
    outlier_labels = detector.fit_predict(X)
    outlier_scores = detector.decision_scores_
    n_outliers = outlier_labels.sum()
    cleaned_df = df.filter(pl.Series(outlier_labels == 0))

    return cleaned_df, outlier_scores, outlier_labels

df_cleared, scores, labels = ecod_clean_dataset(df_cleared)

In [ ]:
df_cleared.shape

(7848, 14)

Как-то так.

Я решил на всякий случай еще немного очистить данные. Например, меня очень смущает несбалансированность классов по фичам, которые я нарек категориальными.

Поэтому возьму и посмотрю еще раз на распределение значений и выкину сэмплы нерепрезентативных классов.

In [ ]:
print(f'Категориальные признаки: {category_like_columns}')
print(f'Вещественные признаки: {key_numeric_features}')

Категориальные признаки: ['Geography', 'Gender', 'Tenure', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Exited']
Вещественные признаки: ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']


Как и обещал, удалим сэмплы маломощных классов по признаку `NumOfProducts`

In [ ]:
print('Before')
print_unique_values_table(df_cleared, 'NumOfProducts')
df_cleared = df_cleared.filter(pl.col('NumOfProducts') <= 2)
print('\nAfter')
print_unique_values_table(df_cleared, 'NumOfProducts')
df_cleared.shape

Before
+---------------+-------+-------------+
| NumOfProducts | count | percent (%) |
+---------------+-------+-------------+
|       1       |  3947 |    50.3%    |
|       2       |  3823 |    48.7%    |
|       3       |   74  |     0.9%    |
|       4       |   4   |     0.1%    |
+---------------+-------+-------------+

After
+---------------+-------+-------------+
| NumOfProducts | count | percent (%) |
+---------------+-------+-------------+
|       1       |  3947 |    50.8%    |
|       2       |  3823 |    49.2%    |
+---------------+-------+-------------+


(7770, 14)

Ну смущает меня эти 0 и 10 лет преданности банку... Все же хочется с равномерным дискретным распределением работать

In [ ]:
print('Before')
print_unique_values_table(df_cleared, 'Tenure')
df_cleared = df_cleared.filter(~pl.col('Tenure').is_in([0.0, 10.0]))
print('\nAfter')
print_unique_values_table(df_cleared, 'Tenure')
df_cleared.shape

Before
+--------+-------+-------------+
| Tenure | count | percent (%) |
+--------+-------+-------------+
|  5.0   |  849  |    10.9%    |
|  7.0   |  835  |    10.7%    |
|  4.0   |  821  |    10.6%    |
|  8.0   |  815  |    10.5%    |
|  2.0   |  805  |    10.4%    |
|  3.0   |  805  |    10.4%    |
|  6.0   |  798  |    10.3%    |
|  1.0   |  761  |     9.8%    |
|  9.0   |  710  |     9.1%    |
|  10.0  |  310  |     4.0%    |
|  0.0   |  261  |     3.4%    |
+--------+-------+-------------+

After
+--------+-------+-------------+
| Tenure | count | percent (%) |
+--------+-------+-------------+
|  5.0   |  849  |    11.8%    |
|  7.0   |  835  |    11.6%    |
|  4.0   |  821  |    11.4%    |
|  8.0   |  815  |    11.3%    |
|  2.0   |  805  |    11.2%    |
|  3.0   |  805  |    11.2%    |
|  6.0   |  798  |    11.1%    |
|  1.0   |  761  |    10.6%    |
|  9.0   |  710  |     9.9%    |
+--------+-------+-------------+


(7199, 14)

In [ ]:
print(f'Raw data shape: {df_pl.shape}')
print(f'Deduplicated cleared data shape: {df_cleared.shape}')

Raw data shape: (10000, 14)
Deduplicated cleared data shape: (7199, 14)


Айдишники убрали из признаков, в попытках увеличить f1-score добавили разные фичи (вроде они сами за себя говорят)

In [ ]:
df_cleared = df_cleared.with_columns([
    pl.when(pl.col("Age") < 30).then(0)
      .when(pl.col("Age") < 45).then(1)
      .when(pl.col("Age") < 60).then(2)
      .otherwise(3).alias("AgeGroup"),

    ((pl.col("Age") >= 60) & (pl.col("Balance") > 0)).alias("SeniorWithBalance"),
    ((pl.col("Age") < 35) & (pl.col("Balance") < 10000)).alias("YoungLowBalance"),
    (pl.col("Balance") / (pl.col("EstimatedSalary") + 1)).alias("BalancetoSalaryRatio"),
    (pl.col("Balance") * pl.col("NumOfProducts")).alias("ClientValueScore"),
    ((pl.col("EstimatedSalary") < 50000) & (pl.col("Balance") > 100000)).alias("LowSalaryHighBalance"),
    ((pl.col("HasCrCard") == 1) & (pl.col("IsActiveMember") == 1) &
     (pl.col("NumOfProducts") == 2)).alias("IdealClientProfile")
])

In [ ]:
category_like_columns += ['AgeGroup', 'SeniorWithBalance', 'YoungLowBalance', 'LowSalaryHighBalance', 'IdealClientProfile']
key_numeric_features += ['BalancetoSalaryRatio', 'ClientValueScore']

In [ ]:
target = 'Exited'
category_like_columns.remove(target)

FEATURES_IN_USE = category_like_columns + key_numeric_features
FEATURES_IN_USE

['Geography',
 'Gender',
 'Tenure',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'AgeGroup',
 'SeniorWithBalance',
 'YoungLowBalance',
 'LowSalaryHighBalance',
 'IdealClientProfile',
 'CreditScore',
 'Age',
 'Balance',
 'EstimatedSalary',
 'BalancetoSalaryRatio',
 'ClientValueScore']

## 4. Кодирование и скалирование

Анализ распределений до этого момента был достаточно сумбурным (да и сейчас ничего не поменяется), но я специально вел отдельно учет категориальных признаков (чтобы применить к ним one-hot encoding), отдельно логарифмированных признаков (чтобы применить к ним скейлинг).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df_normed = df_cleared.select(FEATURES_IN_USE).to_pandas()

numeric_features = key_numeric_features
categorical_features = category_like_columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ]
)

transformed_data = preprocessor.fit_transform(df_normed)

categorical_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
feature_names = np.concatenate([numeric_features, categorical_names])


df_normed = pd.DataFrame(transformed_data, columns=feature_names)
df_normed[target] = df_cleared[target]
df_normed

,CreditScore,Age,Balance,EstimatedSalary,BalancetoSalaryRatio,ClientValueScore,Geography_Germany,Geography_Spain,Gender_Male,Tenure_2.0,...,HasCrCard_1,IsActiveMember_1,AgeGroup_1,AgeGroup_2,AgeGroup_3,SeniorWithBalance_True,YoungLowBalance_True,LowSalaryHighBalance_True,IdealClientProfile_True,Exited
0,0.750000,0.511145,0.997586,-1.228495,0.322964,1.830700,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,-0.348162,-0.560195,0.915354,1.699661,-0.137151,0.339284,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,-0.045607,-0.881597,1.171441,-1.779292,6.409280,2.058828,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
3,0.290565,0.296877,1.230621,-1.756552,4.564649,2.136482,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0
4,1.601636,-1.095865,-1.193378,-0.243984,-0.227946,-1.044229,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7194,0.716383,-0.560195,0.214684,-0.585218,-0.056348,-0.120417,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7195,-0.583483,0.296877,0.315477,0.288134,-0.119758,-0.054288,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7196,0.133684,-0.667329,0.964741,-0.968857,0.151755,0.371686,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1
7197,1.041349,-0.024525,-1.193378,-1.449563,-0.227946,-1.044229,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## 5. Анализ влияния признаков на таргет


In [ ]:
df_normed = pl.from_pandas(df_normed)

In [ ]:
def plot_heatmap(corr_df):
    corr_matrix = corr_df.to_numpy()
    columns = corr_df.columns

    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix, x=columns, y=columns,
        colorscale='RdBu_r', zmin=-1, zmax=1,
    ))
    annotations = [
        dict(x=j, y=i, text=f'{value:.2f}', showarrow=False,
            font=dict(color='white' if abs(value) > 0.5 else 'black', size=10))
        for i, row in enumerate(corr_matrix)
        for j, value in enumerate(row)
    ]

    fig.update_layout(width=1000, height=1000, annotations=annotations)
    fig.show()

plot_heatmap(df_normed.corr())

In [ ]:
FEATURES_IN_USE = df_normed.columns

In [ ]:
correlations = []
for col in FEATURES_IN_USE:
    if col != target:
        try:
            corr = df_normed.select([pl.corr(target, col).alias('correlation')])['correlation'][0]
            correlations.append({'feature': col, 'correlation': abs(corr)})
        except:
            pass

correlation_df = pl.DataFrame(correlations).sort('correlation', descending=True)

print(correlation_df.head(239))
fig = px.bar(correlation_df.to_pandas(),
             x='correlation', y='feature',
             orientation='h', color='correlation',
             color_continuous_scale='Inferno')
fig.update_layout(height=600)
fig.show()

shape: (27, 2)
┌──────────────────────┬─────────────┐
│ feature              ┆ correlation │
│ ---                  ┆ ---         │
│ str                  ┆ f64         │
╞══════════════════════╪═════════════╡
│ AgeGroup_2           ┆ 0.3039      │
│ Age                  ┆ 0.267924    │
│ NumOfProducts_2      ┆ 0.2545      │
│ Geography_Germany    ┆ 0.165281    │
│ AgeGroup_1           ┆ 0.161255    │
│ …                    ┆ …           │
│ Tenure_8.0           ┆ 0.006875    │
│ BalancetoSalaryRatio ┆ 0.004704    │
│ Tenure_5.0           ┆ 0.003776    │
│ Tenure_9.0           ┆ 0.00183     │
│ Tenure_3.0           ┆ 0.001084    │
└──────────────────────┴─────────────┘


Убираем плохо коррелирующие фичи (корреляция $\lt0.05$)

In [ ]:
good_correlation_df = correlation_df.filter(pl.col('correlation') >= 0.05)

good_features = good_correlation_df['feature'].to_list()
good_features.append(target)

df_features = df_normed.select(good_features)
df_features

AgeGroup_2,Age,NumOfProducts_2,Geography_Germany,AgeGroup_1,IdealClientProfile_True,IsActiveMember_1,YoungLowBalance_True,Balance,Gender_Male,SeniorWithBalance_True,Geography_Spain,Exited
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
0.0,0.511145,1.0,1.0,1.0,0.0,0.0,0.0,0.997586,1.0,0.0,0.0,0
0.0,-0.560195,0.0,0.0,1.0,0.0,1.0,0.0,0.915354,0.0,0.0,0.0,0
0.0,-0.881597,1.0,1.0,1.0,0.0,0.0,0.0,1.171441,1.0,0.0,0.0,0
0.0,0.296877,1.0,1.0,1.0,1.0,1.0,0.0,1.230621,0.0,0.0,0.0,0
0.0,-1.095865,1.0,0.0,0.0,0.0,1.0,1.0,-1.193378,0.0,0.0,1.0,0
…,…,…,…,…,…,…,…,…,…,…,…,…
0.0,-0.560195,0.0,0.0,1.0,0.0,1.0,0.0,0.214684,0.0,0.0,0.0,0
0.0,0.296877,0.0,0.0,1.0,0.0,0.0,0.0,0.315477,0.0,0.0,0.0,0
0.0,-0.667329,0.0,1.0,1.0,0.0,0.0,0.0,0.964741,1.0,0.0,0.0,1


Результаты пока неутешительны. Интересно, что именно возраст коррелирует с таргетом в большей степени.

## 6. Обучение эталонных моделей


### 6.1 Разбиение исходных данных на обучающий и контрольный наборы

In [ ]:
df_reg_pd = df_features.clone().to_pandas()

features = [col for col in df_reg_pd.columns if col != target]
X, y = df_reg_pd[features], df_reg_pd[target]

Сейчас в данных присутствует дисбаланс классов сразу по нескольким категориальным переменным, поэтому намеренно не применяем стратификацию, сплитим данные на тренировочную, тестовую и валидационную подвыборки.

Тестовая подвыборка будет составлять 10% от всех сэмплов.
Валидационная - 22% от оставшихся 90% сэмплов.
Использую также стратификацию по таргету, чтобы в обучающую выборку в целом попала такая же доля сэмплов маломощного класса таргета, что и в тестовый поднабор.

In [ ]:
from sklearn.model_selection import train_test_split

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1,
                                                  random_state=seed,
                                                  stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.22,
                                                  random_state=seed,
                                                  stratify=y_temp)

print(f'Train subset: {X_train.shape[0]} ({X_train.shape[0]/X.shape[0]:.2%})')
print(f'Val subset: {X_val.shape[0]} ({X_val.shape[0]/X.shape[0]:.2%})')
print(f'Test subset: {X_test.shape[0]} ({X_test.shape[0]/X.shape[0]:.2%})')

Train subset: 5053 (70.19%)
Val subset: 1426 (19.81%)
Test subset: 720 (10.00%)


### 6.2 Обучение базовой модель без учета дисбаланса

Вот такой подгон порога классификации можно будет использовать чуть позже

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def get_optimal_threshold(model, X, y):
    y_pred_proba = model.predict_proba(X)[:, 1]
    thresholds = np.linspace(0.01, 0.99, 100)
    best_threshold = 0.5
    best_f1 = 0
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        f1 = f1_score(y, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    return best_threshold

In [ ]:
def evaluate_model(model, X, y, model_name='', use_threshold=False, visualize=True):
    threshold = get_optimal_threshold(model, X, y) if use_threshold else 0.5
    y_pred_proba = model.predict_proba(X)[:, 1]
    y_pred = (y_pred_proba >= threshold).astype(int)

    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    auc_score = roc_auc_score(y, y_pred_proba)
    fpr, tpr, thresholds = roc_curve(y, y_pred_proba)
    cm = confusion_matrix(y, y_pred)
    if visualize:
        fig = make_subplots(rows=1, cols=2, subplot_titles=('ROC Curve', 'Metrics Summary'))
        fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines',
                                name=f'ROC (AUC = {auc_score:.3f})'), row=1, col=1)
        fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines',
                                name='Random Classifier', line=dict(dash='dash')), row=1, col=1)

        metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']
        metrics_values = [accuracy, precision, recall, f1, auc_score]
        fig.add_trace(go.Bar(x=metrics_names, y=metrics_values,
                            text=[f'{val:.3f}' for val in metrics_values],
                            textposition='auto'), row=1, col=2)

        fig.update_layout(title=f'{model_name}', height=400, showlegend=True)
        fig.update_xaxes(title_text="False Positive Rate", row=1, col=1)
        fig.update_yaxes(title_text="True Positive Rate", row=1, col=1)
        fig.update_xaxes(title_text="Metrics", row=1, col=2)
        fig.update_yaxes(title_text="Score", range=[0, 1], row=1, col=2)
        fig.show()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': auc_score,
        'confusion_matrix': cm,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

Неужели дошли до запуска обучения моделей, возьмем:

1. Линейные модели:
    * Logistic Regression (Логистическая регрессия)
2. Ансамблевые методы:
    * Random Forest - бэггинг над деревьями решений
    * XGBoost - оптимизированная версия градиентного бустинга


Сразу подсчитаю и визуализирую метрики и ROC-AUC кривую на валидационном поднаборе данных

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

models = {
    'logreg': LogisticRegression(random_state=seed),
    'rf': RandomForestClassifier(random_state=seed),
    'xgb': XGBClassifier(random_state=seed),
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    results[name] = evaluate_model(model, X_val, y_val, name)

А теперь сравним модели между собой...

In [ ]:
def compare_models(results_dict, y_true):
    fig = go.Figure()
    for model_name, metrics in results_dict.items():
        fpr, tpr, _ = roc_curve(y_true, metrics['y_pred_proba'])
        fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines',
                                name=f'{model_name} (AUC = {metrics["roc_auc"]:.3f})'))

    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random Classifier',
                            line=dict(dash='dash', color='gray')))

    fig.update_layout(xaxis_title='False Positive Rate',
                     yaxis_title='True Positive Rate', width=800, height=600)
    fig.show()

    metrics_df = pd.DataFrame(results_dict).T
    metrics_df = metrics_df[['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']]

    return metrics_df

In [ ]:
compare_models(results, y_true=y_val)

,accuracy,precision,recall,f1_score,roc_auc
logreg,0.870968,0.677083,0.298165,0.414013,0.829582
rf,0.864656,0.580645,0.412844,0.482574,0.809929
xgb,0.862553,0.571429,0.40367,0.473118,0.834


Анализ метрик выявил противоречивую картину. Все модели демонстрируют высокую точность предсказания, однако это впечатление обманчиво как раз из-за дисбаланса классов в данных. Если бы мы не обратили внимание на то, как распределен таргет, то и дальше бы радовались при высоких значениях `accuracy`, но вот F1-мера уже более чувствительна к дисбалансу, поскольку требует одновременного достижения хороших значений как `precision`, так и `recall`.

## 7. Оптимизация с учетом дисбаланса

Для начала все же запустимся на взвешенных данных, то есть определим гиперпараметр `class_weight`.

Под капотом:

Классы будут взвешены обратно пропорционально частоте их появления в данных.

Базовая формула для расчёта веса каждого класса: $$\frac{N}{C \cdot n_c}$$

$N$ — общее количество наблюдений \
$C$ — количество классов \
$n_c$ — количество наблюдений в классе



In [ ]:
scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)
weighted_models = {
    'logreg_class_weight': LogisticRegression(random_state=seed, class_weight='balanced'),
    'rf_class_weight': RandomForestClassifier(random_state=seed, class_weight='balanced'),
    'xgb_class_weight': XGBClassifier(random_state=seed, eval_metric='logloss', scale_pos_weight=scale_pos_weight),
}

for name, model in weighted_models.items():
    if name not in results.keys():
        model.fit(X_train, y_train)
        results[name] = evaluate_model(model, X_val, y_val, name, use_threshold=True)
compare_models(results, y_true=y_val)

,accuracy,precision,recall,f1_score,roc_auc
logreg,0.870968,0.677083,0.298165,0.414013,0.829582
rf,0.864656,0.580645,0.412844,0.482574,0.809929
xgb,0.862553,0.571429,0.40367,0.473118,0.834
logreg_class_weight,0.84432,0.492248,0.582569,0.533613,0.827653
rf_class_weight,0.825386,0.441948,0.541284,0.486598,0.8085
xgb_class_weight,0.854137,0.522727,0.527523,0.525114,0.826043


Уже получше, конечно, но все равно по f1 не выходим на нужный нам результат (f1 > 0.59)

Теперь будем пользоваться методами изменения тренировочного набора для решения проблемы дисбаланса классов (over/undersampling).

1. `RandomOverSampler` - случайное увеличение миноритарного класса (может привести к переобучению)

2. `RandomUnderSampler` - случайное уменьшение мажоритарного класса (может терять важную информацию)

3. `SMOTE` (Synthetic Minority Over-sampling Technique) - создание синтетических примеров миноритарного класса

4. `ADASYN` (Adaptive Synthetic Sampling) - адаптивное синтетическое семплирование

5. `BorderlineSMOTE` - пограничное синтетическое семплирование (улучшает качество границы классов)

6. `SMOTEENN` (SMOTE + Edited Nearest Neighbors)

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN, BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN, SMOTETomek

def apply_balancing_methods(X_train, y_train):
    balanced_datasets = {}
    balanced_datasets['original'] = (X_train, y_train)
    methods = [
        ('random_oversampling', RandomOverSampler(random_state=seed)),
        ('random_undersampling', RandomUnderSampler(random_state=seed)),
        ('smote', SMOTE(random_state=seed, k_neighbors=5)),
        ('adasyn', ADASYN(random_state=seed, n_neighbors=5)),
        ('borderline_smote', BorderlineSMOTE(random_state=seed, k_neighbors=5)),
        ('smoteenn', SMOTEENN(random_state=seed)),
    ]

    for method_label, m in methods:
        try:
            print(f'Before {method_label}: {Counter(y_train)}')
            X_b, y_b = m.fit_resample(X_train.copy(), y_train.copy())
            print(f'After {method_label}: {Counter(y_b)}\n')
            balanced_datasets[method_label] = (X_b, y_b)
        except Exception as e:
            print(e)
    return balanced_datasets

balanced_datasets = apply_balancing_methods(X_train, y_train)

Before random_oversampling: Counter({0: 4280, 1: 773})
After random_oversampling: Counter({0: 4280, 1: 4280})

Before random_undersampling: Counter({0: 4280, 1: 773})
After random_undersampling: Counter({0: 773, 1: 773})

Before smote: Counter({0: 4280, 1: 773})
After smote: Counter({0: 4280, 1: 4280})

Before adasyn: Counter({0: 4280, 1: 773})
After adasyn: Counter({1: 4426, 0: 4280})

Before borderline_smote: Counter({0: 4280, 1: 773})
After borderline_smote: Counter({0: 4280, 1: 4280})

Before smoteenn: Counter({0: 4280, 1: 773})
After smoteenn: Counter({0: 2895, 1: 2852})



Запустим модели обучаться на этих данных...

In [ ]:
balanced_models = {
    'logreg': LogisticRegression(random_state=seed, class_weight='balanced'),
    'rf': RandomForestClassifier(random_state=seed, class_weight='balanced'),
    'xgb': XGBClassifier(random_state=seed, eval_metric='logloss'),
}

results_on_test = {}
fitted_models = {}
for balancing_method, balanced_dataset in balanced_datasets.items():
    X_train, y_train = balanced_dataset
    for model_name, balanced_model in balanced_models.items():
        balanced_model_name = f'{model_name}_{balancing_method}'
        if balanced_model_name not in results.keys():
            balanced_model.fit(X_train, y_train)
            results[balanced_model_name] = evaluate_model(balanced_model, X_val, y_val, balanced_model_name, use_threshold=True)
            results_on_test[balanced_model_name] = evaluate_model(balanced_model, X_test, y_test, balanced_model_name, visualize=False)
            fitted_models[balanced_model_name] = balanced_model

In [ ]:
results_df = compare_models(results, y_true=y_val).sort_values('f1_score', ascending=False)
best_model_name = results_df['f1_score'].index[0]
results_df

,accuracy,precision,recall,f1_score,roc_auc
xgb_smoteenn,0.853436,0.516854,0.633028,0.569072,0.845559
xgb_original,0.824684,0.452941,0.706422,0.551971,0.834
rf_smoteenn,0.871669,0.596685,0.495413,0.541353,0.837365
logreg_random_oversampling,0.828191,0.456311,0.646789,0.535104,0.827623
xgb_random_oversampling,0.831697,0.463087,0.633028,0.534884,0.82783
logreg_random_undersampling,0.858345,0.537037,0.53211,0.534562,0.822789
logreg_smoteenn,0.861851,0.55122,0.518349,0.534279,0.823469
logreg_borderline_smote,0.859046,0.539906,0.527523,0.533643,0.825789
logreg_original,0.84432,0.492248,0.582569,0.533613,0.827653
logreg_class_weight,0.84432,0.492248,0.582569,0.533613,0.827653


Смотрим на значения f1, а они все не растут да не растут до 0.59 или выше.

Делать нечего, вроде уже все перепробовал с базовыми моделями и методами решения проблемы несбалансированности таргета.
Запущу на тестовом поднаборе для последней валидации.

In [ ]:
print(f'Best model in terms of f1 score: {best_model_name}')

best_model = fitted_models[best_model_name]
_ = evaluate_model(best_model, X_test, y_test)

Best model in terms of f1 score: xgb_smoteenn


Все равно пока не добился метрики 0.59 и выше на тестовом поднаборе.


И здесь я психанул...

Давайте в целом проверим, можно ли на этих данных построить хороший классификатор, или я просто недостаточно хорошо провел EDA.

Будем пользоваться autoML подходом (не FEDOT, потому что не хочется нести его в лабы).

In [ ]:
!pip install flaml catboost --quiet

In [ ]:
!pip install "numpy<2.0.0"

In [ ]:
from flaml import AutoML
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import numpy as np
import pandas as pd

automl = AutoML()

settings = {
    "time_budget": 300,
    "metric": 'f1',
    "task": 'classification',
    "eval_method": 'cv',
    "n_splits": 5,
    # "use_stratify": True,
    "estimator_list": ['lgbm', 'xgboost', 'catboost', 'rf'],
    "ensemble": True,
    "seed": seed,
}

automl.fit(X_train, y_train, **settings)

[flaml.automl.logger: 10-13 19:38:59] {1752} INFO - task = classification
[flaml.automl.logger: 10-13 19:38:59] {1763} INFO - Evaluation method: cv
[flaml.automl.logger: 10-13 19:38:59] {1862} INFO - Minimizing error metric: 1-f1
[flaml.automl.logger: 10-13 19:38:59] {1979} INFO - List of ML learners in AutoML Run: ['lgbm', 'xgboost', 'catboost', 'rf']
[flaml.automl.logger: 10-13 19:38:59] {2282} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 10-13 19:39:00] {2417} INFO - Estimated sufficient time budget=3742s. Estimated necessary time budget=7s.
[flaml.automl.logger: 10-13 19:39:00] {2466} INFO -  at 0.4s,	estimator lgbm's best error=0.1807,	best estimator lgbm's best error=0.1807
[flaml.automl.logger: 10-13 19:39:00] {2282} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 10-13 19:39:00] {2466} INFO -  at 0.6s,	estimator lgbm's best error=0.1807,	best estimator lgbm's best error=0.1807
[flaml.automl.logger: 10-13 19:39:00] {2282} INFO - iteration 2, curr

In [ ]:
y_pred = automl.predict(X_test)
f1 = f1_score(y_test, y_pred)

print(f'f1 (val): {1 - automl.best_loss:.4f}')
print(f'f1 Score: {f1:.4f}')
print(f'Best model: {automl.best_estimator}')
print(f'Best config: {automl.best_config}')

f1 (val): 0.9849
f1 Score: 0.4881
Best model: rf
Best config: {'n_estimators': 380, 'max_features': 0.14221283508506374, 'max_leaves': 382, 'criterion': 'gini'}


Даже так неудача (видимо, с данными нужно было бы подольше посидеть, фичей напридумывать). Здесь явно переобучились на тренировочной выборке.

## 8. Выводы


Вот и все, конец!

По щеке катится скупая мужская слеза (опять).
Если хотите пронаблюдать, как я впервые пустил слезу, смотрите отчет по лабороторной 1.

Небольшое саммари:

В рамках лабораторной работы я попытался провести комплексный анализ данных, включающий их предобработку, обработку пропущенных значений, дубликатов и аномалий. Все решения для этих этапов принимались из моего опыта в первой лабораторной работе.

Проанализировал также методы борьбы с дисбалансом классов в данных. Попробовал взвешивание классов в датасете, когда модель сильнее штрафуется за ошибки в редком классе, и другие подходы: случайное удаление примеров мажоритарного класса или синтетическое создание примеров миноритарного класса с помощью методов вроде SMOTE.

Но хороших значений f1 так и не удалось получить, несмотря на все приложенные к этому силы.

Часто приходилось строить roc-auc и смотреть на значения f1-score. Кажется, что именно по f1-score лучше сравнивать бейзлайны в случае дисбаланса классов.

**Лабораторная работа выполнена в рамках курса "Машинное обучение" ИТМО, 2025**